### **Nessessary Dependancy**

- python 3.9
- pip upgrade // python -m pip install --upgrade pip
- transformers
- sentencepiece
- tensorflow_addons
- pandas

<h4> << reference >> </h4>

- for fixing fitting Error

https://github.com/hwangtaemin/NLP-with-Naver-movie-review/blob/main/movie_review.ipynb

In [1]:
## install packages

# ! python -m pip install --upgrade pip
# ! pip install transformers
# ! pip install sentencepiece
# ! pip install tensorflow_addons

<center><h2><b>Imports</b><h2></center>

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tqdm import tqdm

from transformers import * # For unstable module import error
import tensorflow_addons as tfa # for optimizer

from source.const import RANDOM_STATE,SEQUENCE_LEN

<center><h2><b>Loading Data</b></h2></center>

In [3]:
# for developing
train = pd.read_csv("./data/ratings_train.txt",sep='\t').sample(frac=0.01,random_state=RANDOM_STATE)
test = pd.read_csv("./data/ratings_test.txt",sep='\t').sample(frac=0.01,random_state=RANDOM_STATE)

print('shape of train : ',train.shape)
print('shape of test : ',test.shape)

shape of train :  (1500, 3)
shape of test :  (500, 3)


<center><h2><b>Preprocessing</b></h2></center>

In [4]:
# get BERT pre-trained model
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased') 

loading file https://huggingface.co/bert-base-multilingual-cased/resolve/main/vocab.txt from cache at C:\Users\kdh/.cache\huggingface\transformers\eff018e45de5364a8368df1f2df3461d506e2a111e9dd50af1fae061cd460ead.6c5b6600e968f4b5e08c86d8891ea99e51537fc2bf251435fb46922e8f7a7b29
loading file https://huggingface.co/bert-base-multilingual-cased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-multilingual-cased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-multilingual-cased/resolve/main/tokenizer_config.json from cache at C:\Users\kdh/.cache\huggingface\transformers\f55e7a2ad4f8d0fff2733b3f79777e1e99247f2e4583703e92ce74453af8c235.ec5c189f89475aac7d8cbd243960a0655cfadc3d0474da8ff2ed0bf1699c2a5f
loading configuration file https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json from cache at C:\Users\kdh/.cache\huggingface\transformers\6c4a5d81a58c9791cdf76a09bce1b5abfb9

In [5]:
# drop na
train.dropna(axis=0,how='any',inplace=True)
test.dropna(axis=0,how='any',inplace=True)

In [6]:
train.loc[:,'label'] = train['label'].astype(int)
test.loc[:,'label'] = test['label'].astype(int)

In [7]:
# huggingface

# convert data into bert input
def get_bert_input(df): # need label
    global tokenizer # do not miss it
    
    ts,segs,msks,lbs = [],[],[],[]
    for i in tqdm(range(df.shape[0])):
        tokenized = tokenizer.encode(df['document'].iloc[i], max_length=SEQUENCE_LEN, truncation=True, padding='max_length')
        zeros =  tokenized.count(0)
        
        segment = [0]*SEQUENCE_LEN
        mask = [1]*(SEQUENCE_LEN-zeros) + [0]*zeros # Non-zero with zero mask
        label = df['label'].iloc[i]
        
        ts.append(tokenized)
        segs.append(segment)
        msks.append(mask)
        lbs.append(label)
    
    ts = np.array(ts)
    segs = np.array(segs)
    msks = np.array(msks)
    lbs = np.array(lbs)
    
    return [ts,segs,msks],lbs

<center><h2><b>Data preparation</b></h2></center>

In [8]:
X_train,y_train = get_bert_input(train)
X_test,y_test = get_bert_input(test)

100%|██████████| 500/500 [00:00<00:00, 5262.89it/s]


<center><h2><b>BERT Modeling</b></h2></center>

#### from
https://huggingface.co/ , https://huggingface.co/bert-base-multilingual-cased

In [9]:
# optimizer
opt = tfa.optimizers.RectifiedAdam(learning_rate=1.0e-5, weight_decay=0.0025, warmup_proportion=0.05)

In [10]:
# modeling
# we gonna use pre-trained model
# bert base multilingual cased model
model = TFBertModel.from_pretrained('bert-base-multilingual-cased') 

# indices must be integer
# input layers
token_inputs = tf.keras.layers.Input((SEQUENCE_LEN,),dtype=tf.dtypes.int32, name='word')
mask_inputs = tf.keras.layers.Input((SEQUENCE_LEN,),dtype=tf.dtypes.int32, name='mask')
segment_inputs = tf.keras.layers.Input((SEQUENCE_LEN,),dtype=tf.dtypes.int32, name='segment')

bert_outputs = model([token_inputs,mask_inputs,segment_inputs])

bert_outputs = bert_outputs[1]
sentiment_first = tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02))(bert_outputs)
sentiment_model = tf.keras.Model([token_inputs, mask_inputs, segment_inputs], sentiment_first)

sentiment_model.compile(optimizer=opt, loss='binary_crossentropy', metrics = ['accuracy'])

sentiment_model.summary()

loading configuration file https://huggingface.co/bert-base-multilingual-cased/resolve/main/config.json from cache at C:\Users\kdh/.cache\huggingface\transformers\6c4a5d81a58c9791cdf76a09bce1b5abfb9cf958aebada51200f4515403e5d08.0fe59f3f4f1335dadeb4bce8b8146199d9083512b50d07323c1c319f96df450c
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 word (InputLayer)              [(None, 128)]        0           []                               
                                                                                                  
 mask (InputLayer)              [(None, 128)]        0           []                               
                                                                                                  
 segment (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  177853440   ['word[0][0]',                   
                                thPoolingAndCrossAt               'mask[0][0]',               

<h4><b>FAIELD</b></h4>

In [11]:
# training
sentiment_model.fit(X_train, y_train, 
                    epochs=4, shuffle=True, batch_size=100, validation_data=(X_test, y_test))

# save weight
sentiment_model.save_weights('./model/huggingface_BERT.h5')

Epoch 1/4


ResourceExhaustedError: Graph execution error:

Detected at node 'model/tf_bert_model/bert/encoder/layer_._1/attention/output/dropout_5/dropout/random_uniform/RandomUniform' defined at (most recent call last):
    File "C:\Users\kdh\miniconda3\envs\test\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\kdh\miniconda3\envs\test\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\kdh\miniconda3\envs\test\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\kdh\miniconda3\envs\test\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\kdh\miniconda3\envs\test\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Users\kdh\miniconda3\envs\test\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.a

<center><h2><b>Evaluation</b></h2></center>